# Тестовое задание: Графовые нейронные сети, классические алгоритмы

## Настройка окружения

```
!conda install -c dglteam dgl
```

_Примечание 1_. Для установки пакета с возможностью обучения моделей на видеокартах необходимо также иметь установленный CUDA.

_Примечание 2_. dgl поддерживает работу с тремя фреймворками: PyTorch 1.5.0+, Apache MXNet 1.6+ и TensorFlow 2.3+. Мы работаем с PyTorch. Полный набор команд для развертывания окружения с возможностью обучения на GPU (cuda версии 10.2):

```cmd
conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
python -c "import torch; print(torch.__version__)"
python -c "import torch; print(torch.version.cuda)"
conda install -c dglteam dgl-cuda10.2
python -m dgl.backend.set_default_backend pytorch
python -c "import dgl; print(dgl.__version__)"
```

DGL (Deep Graph Library)  - это пакет, разработанный для обучения и использования моделей графовых нейронных сетей. В данном пакете реализовано большое количество ставших уже классическими модулей (например, GraphConv, GraphSage и т.д.), на основе которых можно разрабатывать собственные модели. DGL предоставляет большой набор возможностей: работа с GPU, возможности для работы с большими графами и т.д.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np

import dgl
import dgl.nn as gnn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# доп библиотеки
from dgl.data import CoraGraphDataset
import itertools
import scipy

## Работа с графами в `dgl`

В `dgl` граф - объект класс `DGLGraph`. Для его создания нужно указать:
* тензор начальных узлов для ребер
* тензор конечных узлов для ребер
* кол-во вершин (узлы нумеруются, начиная с 0; кол-во вершин можно не задавать, если они все перечислены в списках для создания ребер)

В `dgl` граф всегда ориентированный.

На узлах (`.ndata`) и ребрах (`.edata`) могут храниться атрибуты. Их особенности:
* только числовые тензоры;
* атрибуты всех узлов (ребер) должны иметь одинаковый размер;
* каждый атрибут имеет уникальное имя (атрибуты узлов и ребер могут иметь одинаковые имена);
* первая размерность тензора атрибутов должна быть равна кол-ву узлов (ребер);
* срез по строкам возвращает атрибуты одного узла (ребра)

Пример создания графа:
```python
G = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
G.ndata['x'] = torch.randn(6, 3)
G.ndata['y'] = torch.randint(0, 2, (6, ))
```

Кроме возможностей по созданию наборов данных, `dgl` предоставляет большой набор готовых датасетов в `dgl.data`.

`DGLGraph` имеют большое количество свойств и методов для работы с графовыми структурами, которые можно найти в [документации](https://docs.dgl.ai/api/python/dgl.DGLGraph.html)

### Задание 1. 

Загрузите датасет `CoraGraphDataset` из `dgl.data`. Этот датасет состоит из одного графа. Выведите на экран:

* количество узлов в графе;
* количество ребер в графе;
* размерность атрибутов на узлах;
* количество классов узлов в датасете

Выделите подграф (`dgl.node_subgraph`), содержащий узлы, относящиеся к трем наиболее часто встречающимся классам. __1 балл__


In [3]:
dataset = CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [4]:
dict_label = {k:v for k, v in zip(g.ndata['label'].unique(),
                                [(g.ndata['label'] == x_u).sum() for x_u in g.ndata['label'].unique()]
                               )}

In [5]:
dict_label

{tensor(0): tensor(351),
 tensor(1): tensor(217),
 tensor(2): tensor(418),
 tensor(3): tensor(818),
 tensor(4): tensor(426),
 tensor(5): tensor(298),
 tensor(6): tensor(180)}

In [6]:
dgl.node_subgraph(g, [2,3,4])

Graph(num_nodes=3, num_edges=4,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'__orig__': Scheme(shape=(), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

### Задание 2. 

Файл `cities_nodes.csv` содержит данные о городах: название, страна, население, дату основания и координаты. В файле `cities_edges.csv` содержатся данные о связях между городам: города соединяются ребрами в случае, если в статье русскоязычной Википедии о городе присутсвует ссылка на страницу для другого города. Создайте на основе двух этих файлов `dgl.graph`. Для этого:


1. занумеруйте города (узлы) целыми числами;

2. представьте каждое ребро в виде пары двух целых чисел;

3. в качестве атрибутов (`g.ndata['feat']`) узлов используйте логарифм численности населения в городе и входящую степень узлов (результат сохраните в виде двумерного тензора размера `(n_nodes, 2)` и приведите к `torch.float32`);

4. занумеруйте страны целыми числами и в качестве меток узлов (`g.ndata['label']`) используйте номер соотвествующей страны; используйте тензор типа `torch.int64`;

5. в качестве атрибутов (`g.edata['distance']`) ребер используйте логарифм расстояния между городами (результат сохраните в виде одномерного тензора размера `(n_edges, )` и приведите к `torch.float32`);

6. добавьте графу атрибут `num_classes`, в котором хранится число классов (стран);

7. разбейте множество узлов на обучающее и валидационное множество в соотношении 90%-10%. Для представления каждого из множеств создайте булев тензор $t$ размерности, равной количеству узлов в графе, где $t[i]==True$, если узел $i$ входит в соответствующее множество. Добавьте в словарь `ndata` два ключа `train_mask`, `val_mask`. __(1 балл)__


In [7]:
df_nodes = pd.read_csv('cities_nodes.csv')
df_edges = pd.read_csv('cities_edges.csv')
df_nodes.head()

,Unnamed: 0,country_label,population,inception,lat,long
0,Минск,Белоруссия,2009786,01.01.1067,27.561837,53.902246
1,Гомель,Белоруссия,510300,01.01.1142,30.983333,52.441667
2,Брест,Белоруссия,340318,01.01.1017,23.656944,52.084722
3,Гродно,Белоруссия,356900,01.01.1128,23.816667,53.666667
4,Вилейка,Белоруссия,27167,25.11.1460,26.916667,54.483333


In [8]:
# создадим словарь {'название_города', номер} 
city_dir = {k:v for k, v in zip(df_nodes['Unnamed: 0'],
                                [i for i in range(0, len(df_nodes['Unnamed: 0']))]
                               )}
# создадим граф
g = dgl.graph((torch.from_numpy(df_edges.source.map(city_dir).values), 
               torch.from_numpy(df_edges.target.map(city_dir).values)), num_nodes=315)

# присвоим атрибутам узлов feat нужные значения
g.ndata['feat'] = torch.stack((torch.log(torch.from_numpy(df_nodes.population.values)), \
                               g.in_degrees()), dim = 1).type(torch.float32)


# создадим словарь {'навзание_страны', номер}
country_dir = {k:v for k, v in zip(df_nodes.country_label.unique(),
                                [i for i in range(0, len(df_nodes.country_label.value_counts()))]
                               )}

# присвоим атрибутам узлов label нужные значения
g.ndata['label'] = torch.from_numpy(df_nodes.country_label.map(country_dir).values).type(torch.int64)

# присвоим атрибутам ребер distance нужные значения
g.edata['distance'] = torch.log(torch.from_numpy(df_edges.distance.values)).type(torch.float32)

# присвоим атрибутам узлов num_classes нужные значения
g.ndata['num_classes'] = torch.ones(315) * 4

# разобъем выборку на train и val
n_nodes = df_nodes.shape[0]
n_train = int(n_nodes * 0.9)
n_val = int(n_nodes * 0.1)
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
train_mask[:n_train] = True
val_mask[n_train:] = True

# присвоим атрибутам узлов train_mask и val_mask нужные значения
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask

## Построение нейронных сетей с использованием `dgl`

In [9]:
g.edata['distance'].shape

torch.Size([12666])

`dgl` предоставляет [большое количество](https://docs.dgl.ai/api/python/nn.html) модулей-блоков для построения нейронных сетей. С точки зрения реализации они являются наследниками стандартного `torch.nn.Module`. Поэтому общая логика написания сети остается точно такой же, как и при использовании `torch`: вы описываете модель при помощи класса-наследника `torch.nn.Module`, определяете в методе `__init__` все необходимые блоки и реализуете метод `forward`, где вызываете модули в нужной последовательности. Кроме модулей графовых сетей вы можете использовать и "классические": `Linear`, `Dropout` и т.д.

В отличие от модулей из `torch`, где метод `forward` обычно ожидает один параметр на входе (как `Linear` ожидает один тензор), модули из `dgl` обычно ожидают 2 параметра: граф и матрицу атрибутов узлов. Конкретный модуль может ожидать и чего-то другого, поэтому рекомендуется сверяться с документацией.

Пример работы с модулями из `dgl`

```python
import dgl.nn as gnn
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
g = dgl.add_reverse_edges(g)
g.ndata['x'] = torch.randn(6, 3)

conv = gnn.GraphConv(3, 32)
h = conv(g, g.ndata['x'])
h.shape # h - тензор скрытых представлений узлов после свертки conv
```

Пример описания графовой нейронной сети при помощи `dgl`
```python
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
```

### Задание 3. 
Используя граф, полученный в задании 2, и графовую нейронную сеть, написанную с использованием библиотеки `dgl`, решите задачу классификации городов по странам (задача классификации узлов). __(2 балла)__

На каждой эпохи обучения выводите на экран следующую информацию:
1. Номер эпохи
2. Значение функции потерь на обучающем множестве
3. Значение метрики accuracy (`sklearn.metrics.accuracy_score`) на обучающем множестве
3. Значение метрики accuracy (`sklearn.metrics.accuracy_score`) на тестовом множестве

Важно: при расчете значения функции потерь при обучении используйте только узлы, относящиеся к обучающему множеству! Узлы, относящиеся к валидационному множеству, для простоты можно оставить в графе, но прогнозы для этих узлов не должны оказывать влияние на функцию потерь и обратное распространение ошибки при обучении.

In [11]:
import sklearn
from sklearn.metrics import accuracy_score

In [12]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [13]:
model = GCN(g.ndata['feat'].shape[1], 16, len(df_nodes.country_label.value_counts()))

In [14]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    for e in range(100):
        logits = model(g, features)

        pred = logits.argmax(1)
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

#         train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        train_acc = accuracy_score(pred[train_mask], labels[train_mask])
#         val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        val_acc = accuracy_score(pred[val_mask], labels[val_mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('In epoch {}, loss: {:.3f}, train acc: {:.3f}, val acc: {:.3f}'.format(
            e, loss, train_acc, val_acc))

train(g, model)

In epoch 0, loss: 4.377, train acc: 0.251, val acc: 0.000
In epoch 1, loss: 3.427, train acc: 0.226, val acc: 0.000
In epoch 2, loss: 2.806, train acc: 0.226, val acc: 0.000
In epoch 3, loss: 2.270, train acc: 0.230, val acc: 0.000
In epoch 4, loss: 2.214, train acc: 0.276, val acc: 0.000
In epoch 5, loss: 2.284, train acc: 0.279, val acc: 0.062
In epoch 6, loss: 2.118, train acc: 0.279, val acc: 0.094
In epoch 7, loss: 1.822, train acc: 0.283, val acc: 0.312
In epoch 8, loss: 1.576, train acc: 0.297, val acc: 0.406
In epoch 9, loss: 1.503, train acc: 0.071, val acc: 0.875
In epoch 10, loss: 1.515, train acc: 0.085, val acc: 0.969
In epoch 11, loss: 1.502, train acc: 0.307, val acc: 0.969
In epoch 12, loss: 1.435, train acc: 0.307, val acc: 0.969
In epoch 13, loss: 1.320, train acc: 0.311, val acc: 0.969
In epoch 14, loss: 1.180, train acc: 0.314, val acc: 0.969
In epoch 15, loss: 1.062, train acc: 0.318, val acc: 0.969
In epoch 16, loss: 1.009, train acc: 0.364, val acc: 0.969
In epoc

### Задание 4. 
Используя граф, полученный в задании 2, и графовую нейронную сеть, написанную с использованием библиотеки `dgl`, решите задачу предсказания связей. __(2 балла)__

Цикл обучения должен состоять из следующих шагов:
1. получение представлений `h` узлов при помощи сети, аналогичной заданию 3;
2. конструирование графа отрицательных примеров (см функцию `construct_negative_graph`);
3. расчет прогнозов ребер на основе исходного графа и скрытых представлений узлов `h` (см класс `MLPPredictor`) - для этих ребер модель должна предсказывать метку 1;
4. расчет прогнозов ребер на основе графа отрицательных примеров и и скрытых представлений узлов `h` (см класс `MLPPredictor`) - для этих ребер модель должна предсказывать метку 0;
5. расчет функции потерь (например, `CrossEntropyLoss`) и шаг оптимизации.

На каждой эпохи обучения выводите на экран следующую информацию:
1. Номер эпохи;
2. Значение функции потерь на обучающем множестве.

Примечания: 
1. для простоты в данной задаче не предполагается разбиение ребер на обучающее и тестовое множество;
2. на каждой эпохе получение эмбеддингов `h` происходит только 1 раз на основе исходного графа;

In [15]:
# prepare data for training and validation set

u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)

train_pos_u, train_pos_v = u[eids], v[eids]

def construct_negative_graph(g, k):
    '''Берет ребра из графа g и случайным образом заменяет вершины на концах ребер'''
    u, v = g.edges()
    neg_u = u.repeat_interleave(k).long()
    neg_v = torch.randint(0, g.num_nodes(), (len(neg_u),)).long()
    return dgl.graph((neg_u, neg_v), num_nodes=g.num_nodes())

neg_u, neg_v = construct_negative_graph(g, 1).edges()
neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
train_neg_u, train_neg_v = neg_u[neg_eids], neg_v[neg_eids]

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

In [16]:
from dgl.nn import SAGEConv

# создадим модель
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
class MLPPredictor(nn.Module):
    '''Берет представления узлов, находящихся на концах ребер, конкатенирует их и прогоняет через небольшую полносвязную
    сеть для получения прогноза вероятности существования ребра'''
    def __init__(self, h_feats, out_feats=1):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, out_feats)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [17]:
train_g = dgl.remove_edges(g, eids)

In [18]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
pred = MLPPredictor(16)

In [19]:
# зададим оптимизатор
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# обучим модель 
all_logits = []
for e in range(100):
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    loss = F.binary_cross_entropy_with_logits(scores, labels)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 6.4814372062683105
In epoch 1, loss: 1.5469502210617065
In epoch 2, loss: 4.237903594970703
In epoch 3, loss: 4.325287818908691
In epoch 4, loss: 2.874101400375366
In epoch 5, loss: 1.0443806648254395
In epoch 6, loss: 1.0991381406784058
In epoch 7, loss: 1.8753777742385864
In epoch 8, loss: 2.0923545360565186
In epoch 9, loss: 1.9056904315948486
In epoch 10, loss: 1.487423062324524
In epoch 11, loss: 0.997351348400116
In epoch 12, loss: 0.6487745642662048
In epoch 13, loss: 0.6909669041633606
In epoch 14, loss: 0.9358013272285461
In epoch 15, loss: 1.0631556510925293
In epoch 16, loss: 1.0176939964294434
In epoch 17, loss: 0.8556585907936096
In epoch 18, loss: 0.6759239435195923
In epoch 19, loss: 0.5824633240699768
In epoch 20, loss: 0.587801456451416
In epoch 21, loss: 0.6367713212966919
In epoch 22, loss: 0.6764504909515381
In epoch 23, loss: 0.6747308373451233
In epoch 24, loss: 0.6343117952346802
In epoch 25, loss: 0.6289168000221252
In epoch 26, loss: 0.6689729

In [20]:
# model = GC(g.ndata['feat'].shape[1], 16, len(df_nodes.country_label.value_counts()))
# train_classifier(g, model)

### Задание 5. 

Используя граф, полученный в задании 2, и графовую нейронную сеть, написанную с использованием библиотеки `dgl`, решите задачу предсказания расстояния между городами (задача регрессии на ребрах). __(2 балла)__

Цикл обучения должен состоять из следующих шагов:
1. получение представлений `h` узлов при помощи сети, аналогичной заданию 3;
3. расчет прогнозов ребер на основе исходного графа и скрытых представлений узлов `h` (см класс `MLPPredictor`);
5. расчет функции потерь (например, `MSELoss`) и шаг оптимизации.

На каждой эпохи обучения выводите на экран следующую информацию:
1. Номер эпохи
2. Значение функции потерь на обучающем множестве.

Примечание: для простоты в данной задаче не предполагается разбиение ребер на обучающее и тестовое множество.

In [22]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.sage = GCN(in_features, hidden_features, out_features)
        self.pred = MLPPredictor(5)
    def forward(self, g, x):
        h = self.sage(g, x)
        return self.pred(g, h)

In [23]:
node_features = g.ndata['feat']
edge_label = g.edata['distance']

model = Model(2, 20, 5)
opt = torch.optim.Adam(model.parameters())
for epoch in range(100):
    pred = model(g, node_features)
    loss = ((pred - edge_label) ** 2).mean()
    opt.zero_grad()
    loss.backward()
    opt.step()
    print('In epoch {}, loss: {}'.format(epoch, loss))

In epoch 0, loss: inf
In epoch 1, loss: nan
In epoch 2, loss: nan
In epoch 3, loss: nan
In epoch 4, loss: nan
In epoch 5, loss: nan
In epoch 6, loss: nan
In epoch 7, loss: nan
In epoch 8, loss: nan
In epoch 9, loss: nan
In epoch 10, loss: nan
In epoch 11, loss: nan
In epoch 12, loss: nan
In epoch 13, loss: nan
In epoch 14, loss: nan
In epoch 15, loss: nan
In epoch 16, loss: nan
In epoch 17, loss: nan
In epoch 18, loss: nan
In epoch 19, loss: nan
In epoch 20, loss: nan
In epoch 21, loss: nan
In epoch 22, loss: nan
In epoch 23, loss: nan
In epoch 24, loss: nan
In epoch 25, loss: nan
In epoch 26, loss: nan
In epoch 27, loss: nan
In epoch 28, loss: nan
In epoch 29, loss: nan
In epoch 30, loss: nan
In epoch 31, loss: nan
In epoch 32, loss: nan
In epoch 33, loss: nan
In epoch 34, loss: nan
In epoch 35, loss: nan
In epoch 36, loss: nan
In epoch 37, loss: nan
In epoch 38, loss: nan
In epoch 39, loss: nan
In epoch 40, loss: nan
In epoch 41, loss: nan
In epoch 42, loss: nan
In epoch 43, loss: na

### Задание 6. 
Сведите эмбеддинги узлов, полученные при обучении модели в задаче 5, к векторам размерности 2, используя метод главных компонент (`sklearn.decomposition.PCA`). Визуализируйте сеть, используя теперь для координат точек векторное представление узлов размерности 2. Покажите цветом узла страну, к которой относится город. __(1 балл)__ 

Для преобразования `DGLGraph` к `nx.Graph` можно воспользоваться функцией `dgl.to_networkx`.